In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
#Read train and test csv
train = pd.read_csv("train.csv") #label,pixel0,pixel1,...,pixel783
test_x = pd.read_csv("test.csv") #pixel0,pixel1,...,pixel783 (No Label)
train_y=train['label'] #Our target only the labels 
train_x=train.drop('label','columns') #Our source only the pixels, we drop the column 'label'

In [3]:
#Separate the first 2000 for training, the remain for validation
train_x,dev_x=train_x[2000:],train_x[:2000] 
train_y,dev_y=train_y[2000:],train_y[:2000]

In [4]:
#Data shape
print(train_x.shape,train_y.shape,dev_x.shape,dev_y.shape)

(40000, 784) (40000,) (2000, 784) (2000,)


In [5]:
#Pandas to numpy
train_x=train_x.values
train_y=train_y.values
dev_x=dev_x.values
dev_y=dev_y.values
#Transform targets [0,9,...,7] to [[1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,1],...,[0,0,0,0,0,0,0,1,0,0]]
train_y = tf.keras.utils.to_categorical(train_y, 10)
dev_y = tf.keras.utils.to_categorical(dev_y, 10)

In [6]:
#Create the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Reshape((28,28,1), input_shape=(784,))) #Reshape input [Pixel0,...,Pixel783] to [Pixel0,.......,Pixel27]
                                                                                                        # [......................]
                                                                                                        # [......................]
                                                                                                        # [Pixel256,...,Pixel783]    
model.add(tf.keras.layers.BatchNormalization()) #BachNorm
model.add(tf.keras.layers.Activation("relu"))   #Relu Activation

model.add(tf.keras.layers.Conv2D(64, kernel_size=(5, 5), strides=(2,2))) #2D Convolution Layer
model.add(tf.keras.layers.BatchNormalization()) #BachNorm
model.add(tf.keras.layers.Activation("relu")) #Relu Activation

model.add(tf.keras.layers.Conv2D(128, kernel_size=(5, 5), strides=(2,2))) #2D Convolution Layer
model.add(tf.keras.layers.BatchNormalization()) #BachNorm
model.add(tf.keras.layers.Activation("relu")) #Relu Activation

model.add(tf.keras.layers.Conv2D(256, (3, 3), strides=(2,2))) #2D Convolution Layer
model.add(tf.keras.layers.Dropout(0.5)) #Dropout
model.add(tf.keras.layers.Flatten()) #Flatten the 'squared' matrix to a (1,784) vector 
model.add(tf.keras.layers.Dense(128,activation='relu')) #Dense Layer with relu activation
model.add(tf.keras.layers.Dense(10, activation='softmax')) #Dense Layer with softmax activation so it can predict one of the 10 Labels

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 1)         4         
_________________________________________________________________
activation (Activation)      (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 12, 12, 64)        1664      
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 12, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 128)         204928    
__________

In [7]:
#Train the model
model.fit(train_x, train_y,
          batch_size=1000,
          epochs=10,
          verbose=1,
          validation_data=(dev_x, dev_y))

Train on 40000 samples, validate on 2000 samples
Epoch 1/10
40000/40000 [==============================] - 17s 435us/step - loss: 0.6239 - acc: 0.8052 - val_loss: 0.8054 - val_acc: 0.9245
Epoch 2/10
40000/40000 [==============================] - 17s 418us/step - loss: 0.1692 - acc: 0.9488 - val_loss: 0.7290 - val_acc: 0.8650
Epoch 3/10
40000/40000 [==============================] - 17s 424us/step - loss: 0.1248 - acc: 0.9613 - val_loss: 0.5080 - val_acc: 0.8955
Epoch 4/10
40000/40000 [==============================] - 17s 419us/step - loss: 0.0864 - acc: 0.9735 - val_loss: 0.5865 - val_acc: 0.8035
Epoch 5/10
40000/40000 [==============================] - 17s 414us/step - loss: 0.0747 - acc: 0.9767 - val_loss: 0.4800 - val_acc: 0.8295
Epoch 6/10
40000/40000 [==============================] - 17s 417us/step - loss: 0.0523 - acc: 0.9843 - val_loss: 0.1603 - val_acc: 0.9520
Epoch 7/10
40000/40000 [==============================] - 17s 415us/step - loss: 0.0576 - acc: 0.9809 - val_loss: 0.1

In [9]:
#Predict on the test set
prediction=model.predict(test_x)

In [10]:
#Save results
final=pd.DataFrame(np.array(prediction.argmax(axis=1)),columns=['Label'])
final.index += 1 
final.index.names = ['ImageId']
final.to_csv("result.csv")